In [5]:
#!/usr/bin/env python3
import os
import sys
from pathlib import Path

def list_files_in_current_directory():
    """Wylistuje wszystkie pliki w folderze, w którym znajduje się skrypt."""
    
    # Pobierz ścieżkę do folderu ze skryptem
    try:
        script_dir = Path(__file__).parent.absolute()
    except NameError:
        # Jeśli __file__ nie jest zdefiniowane, użyj bieżącego folderu
        script_dir = Path.cwd()
    
    print(f"Folder: {script_dir}")
    print("-" * 50)
    
    # Liczniki
    file_count = 0
    folder_count = 0
    
    try:
        # Iteruj przez wszystkie elementy w folderze
        for item in sorted(script_dir.iterdir()):
            if item.is_file():
                file_size = item.stat().st_size
                # Formatuj rozmiar pliku
                if file_size < 1024:
                    size_str = f"{file_size} B"
                elif file_size < 1024**2:
                    size_str = f"{file_size/1024:.1f} KB"
                elif file_size < 1024**3:
                    size_str = f"{file_size/(1024**2):.1f} MB"
                else:
                    size_str = f"{file_size/(1024**3):.1f} GB"
                
                print(f"[PLIK] {item.name} ({size_str})")
                file_count += 1
                
            elif item.is_dir():
                print(f"[FOLDER] {item.name}/")
                folder_count += 1
    
    except PermissionError:
        print("Błąd: Brak uprawnień do odczytu folderu")
        return
    except Exception as e:
        print(f"Błąd: {e}")
        return
    
    print("-" * 50)
    print(f"Znaleziono: {file_count} plików, {folder_count} folderów")

def list_files_recursively():
    """Wylistuje wszystkie pliki rekursywnie (z podfolderami)."""
    
    try:
        script_dir = Path(__file__).parent.absolute()
    except NameError:
        # Jeśli __file__ nie jest zdefiniowane, użyj bieżącego folderu
        script_dir = Path.cwd()
    
    print(f"Folder (rekursywnie): {script_dir}")
    print("-" * 50)
    
    file_count = 0
    
    try:
        # Rekursywnie przeszukaj wszystkie pliki
        for item in sorted(script_dir.rglob('*')):
            if item.is_file():
                # Oblicz relatywną ścieżkę
                rel_path = item.relative_to(script_dir)
                file_size = item.stat().st_size
                
                if file_size < 1024:
                    size_str = f"{file_size} B"
                elif file_size < 1024**2:
                    size_str = f"{file_size/1024:.1f} KB"
                elif file_size < 1024**3:
                    size_str = f"{file_size/(1024**2):.1f} MB"
                else:
                    size_str = f"{file_size/(1024**3):.1f} GB"
                
                print(f"{rel_path} ({size_str})")
                file_count += 1
                
    except PermissionError:
        print("Błąd: Brak uprawnień do odczytu niektórych folderów")
        return
    except Exception as e:
        print(f"Błąd: {e}")
        return
    
    print("-" * 50)
    print(f"Znaleziono łącznie: {file_count} plików")

if __name__ == "__main__":
    print("=== LISTA PLIKÓW ===\n")
    
    # Sprawdź argumenty
    if len(sys.argv) > 1 and sys.argv[1] == "-r":
        list_files_recursively()
    else:
        list_files_in_current_directory()
        print("\nUżycie: python skrypt.py [-r]")
        print("  -r : wylistuj pliki rekursywnie (z podfolderami)")

=== LISTA PLIKÓW ===

Folder: \\nas6efab6\Public\Wasalski_repo_mentoring
--------------------------------------------------
[PLIK] .DS_Store (6.0 KB)
[FOLDER] .git/
[FOLDER] azure_integrator/
[PLIK] classess.ipynb (2.1 KB)
[FOLDER] dbt/
[FOLDER] expense-tracker-docker/
[PLIK] fake_data.ipynb (33.8 KB)
[FOLDER] jira_api/
[PLIK] listing_all_files.ipynb (7.1 KB)
[FOLDER] machine learning mastery/
[FOLDER] Topics/
--------------------------------------------------
Znaleziono: 4 plików, 7 folderów

Użycie: python skrypt.py [-r]
  -r : wylistuj pliki rekursywnie (z podfolderami)


In [6]:
all_files = list_files_in_current_directory()
all_files

Folder: \\nas6efab6\Public\Wasalski_repo_mentoring
--------------------------------------------------
[PLIK] .DS_Store (6.0 KB)
[FOLDER] .git/
[FOLDER] azure_integrator/
[PLIK] classess.ipynb (2.1 KB)
[FOLDER] dbt/
[FOLDER] expense-tracker-docker/
[PLIK] fake_data.ipynb (33.8 KB)
[FOLDER] jira_api/
[PLIK] listing_all_files.ipynb (7.1 KB)
[FOLDER] machine learning mastery/
[FOLDER] Topics/
--------------------------------------------------
Znaleziono: 4 plików, 7 folderów
